# VLA Fine-Tuning: SmolVLA on Meta-World

This notebook fine-tunes [SmolVLA](https://huggingface.co/lerobot/smolvla_base) (450M-param Vision-Language-Action model) on [Meta-World](https://github.com/Farama-Foundation/Metaworld) manipulation tasks.

We run a **3-way comparison**:
1. **Random init** — train SmolVLA from scratch (architecture only, no pretrained weights)
2. **Pretrained zero-shot** — evaluate the pretrained SmolVLA directly (no fine-tuning)
3. **Fine-tuned** — adapt pretrained SmolVLA to Meta-World tasks

**Requirements**: Colab Pro+ with A100 GPU. Training takes ~5 hours per run.

## 1. Setup: Install Conda + LeRobot + SmolVLA

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()
# NOTE: Runtime will restart automatically after this cell. That's expected.

In [ ]:
!git clone https://github.com/huggingface/lerobot.git
!conda install ffmpeg=7.1.1 -c conda-forge -y
!cd lerobot && pip install -e ".[smolvla]"
!pip install "gymnasium==1.1.0" metaworld wandb matplotlib

## 2. (Optional) Login to Weights & Biases for training curves

In [ ]:
import wandb
wandb.login()

## 3. Pick Meta-World tasks

We select 3 diverse tasks from the MT50 suite:
- `assembly-v3` — insert a peg into a hole
- `dial-turn-v3` — turn a dial
- `handle-press-side-v3` — press a handle from the side

These test different manipulation skills: insertion, rotation, and pressing.

In [ ]:
TASKS = "assembly-v3,dial-turn-v3,handle-press-side-v3"
DATASET = "lerobot/metaworld_mt50"
STEPS_FINETUNE = 20000
STEPS_SCRATCH = 20000
BATCH_SIZE = 64
EVAL_FREQ = 2000
EVAL_EPISODES = 5
OUTPUT_BASE = "/content/outputs"

## 4. Run 1: Fine-tune pretrained SmolVLA

Start from `lerobot/smolvla_base` (pretrained on real SO100 robot data) and adapt to Meta-World.
This is the standard VLA workflow — take a foundation model, fine-tune on your target domain.

In [ ]:
!cd lerobot && python lerobot/scripts/train.py \
    --policy.path=lerobot/smolvla_base \
    --dataset.repo_id={DATASET} \
    --env.type=metaworld \
    --env.task={TASKS} \
    --batch_size={BATCH_SIZE} \
    --steps={STEPS_FINETUNE} \
    --eval.n_episodes={EVAL_EPISODES} \
    --eval_freq={EVAL_FREQ} \
    --save_freq={EVAL_FREQ} \
    --output_dir={OUTPUT_BASE}/finetuned \
    --job_name=smolvla_finetuned \
    --policy.device=cuda \
    --wandb.enable=true

## 5. Run 2: Train from scratch (random init)

Same architecture, same training, but **no pretrained weights**. This shows how much the pretrained VLM features help.

In [ ]:
!cd lerobot && python lerobot/scripts/train.py \
    --policy.type=smolvla \
    --dataset.repo_id={DATASET} \
    --env.type=metaworld \
    --env.task={TASKS} \
    --batch_size={BATCH_SIZE} \
    --steps={STEPS_SCRATCH} \
    --eval.n_episodes={EVAL_EPISODES} \
    --eval_freq={EVAL_FREQ} \
    --save_freq={EVAL_FREQ} \
    --output_dir={OUTPUT_BASE}/scratch \
    --job_name=smolvla_scratch \
    --policy.device=cuda \
    --wandb.enable=true

## 6. Evaluate all 3 conditions

Run evaluation on each task for:
- **Pretrained zero-shot** (no training on Meta-World)
- **Fine-tuned** (best checkpoint)
- **From scratch** (best checkpoint)

In [ ]:
import subprocess, json, os

conditions = {
    "pretrained_zeroshot": "lerobot/smolvla_base",
    "finetuned": f"{OUTPUT_BASE}/finetuned/checkpoints/last/pretrained_model",
    "from_scratch": f"{OUTPUT_BASE}/scratch/checkpoints/last/pretrained_model",
}

tasks = TASKS.split(",")
results = {}

for cond_name, policy_path in conditions.items():
    if not os.path.exists(policy_path) and "/" not in policy_path[:5]:
        print(f"Skipping {cond_name}: checkpoint not found at {policy_path}")
        continue
    results[cond_name] = {}
    for task in tasks:
        print(f"\n--- Evaluating {cond_name} on {task} ---")
        eval_dir = f"{OUTPUT_BASE}/eval/{cond_name}/{task}"
        cmd = [
            "python", "lerobot/scripts/eval.py",
            f"--policy.path={policy_path}",
            "--env.type=metaworld",
            f"--env.task={task}",
            "--eval.batch_size=1",
            "--eval.n_episodes=10",
            f"--output_dir={eval_dir}",
        ]
        result = subprocess.run(cmd, capture_output=True, text=True, cwd="/content/lerobot")
        print(result.stdout[-500:] if result.stdout else "No stdout")
        if result.returncode != 0:
            print(f"Error: {result.stderr[-500:]}")
            results[cond_name][task] = 0.0
        else:
            # Parse success rate from output (adjust parsing based on actual output format)
            results[cond_name][task] = "check eval_dir for results"

print("\n=== All Results ===")
for cond, task_results in results.items():
    print(f"\n{cond}:")
    for task, score in task_results.items():
        print(f"  {task}: {score}")

## 7. Visualize training curves

In [ ]:
import matplotlib.pyplot as plt
import json, glob, os
import numpy as np

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for idx, (label, output_dir) in enumerate([
    ("Fine-tuned (pretrained init)", f"{OUTPUT_BASE}/finetuned"),
    ("From scratch (random init)", f"{OUTPUT_BASE}/scratch"),
]):
    log_file = os.path.join(output_dir, "training_log.json")
    if not os.path.exists(log_file):
        # Try reading from W&B or CSV logs instead
        log_files = glob.glob(os.path.join(output_dir, "**/*.json"), recursive=True)
        print(f"Available log files for {label}: {log_files}")
        continue

    with open(log_file) as f:
        logs = [json.loads(line) for line in f]

    steps = [l["step"] for l in logs if "loss" in l]
    losses = [l["loss"] for l in logs if "loss" in l]

    color = "tab:blue" if idx == 0 else "tab:orange"
    axes[0].plot(steps, losses, label=label, color=color, alpha=0.8)

axes[0].set_xlabel("Training Steps")
axes[0].set_ylabel("Loss")
axes[0].set_title("Training Loss: Fine-tuned vs From Scratch")
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Bar chart for success rates (fill in after eval)
# Placeholder data — replace with actual eval results
ax2 = axes[1]
task_names = ["assembly", "dial-turn", "handle-press"]
x = np.arange(len(task_names))
width = 0.25

# Replace these with actual success rates from evaluation
scratch_rates = [0.0, 0.0, 0.0]  # Update after eval
zeroshot_rates = [0.0, 0.0, 0.0]  # Update after eval
finetuned_rates = [0.0, 0.0, 0.0]  # Update after eval

ax2.bar(x - width, scratch_rates, width, label="From scratch", color="tab:red", alpha=0.8)
ax2.bar(x, zeroshot_rates, width, label="Pretrained (zero-shot)", color="tab:orange", alpha=0.8)
ax2.bar(x + width, finetuned_rates, width, label="Fine-tuned", color="tab:green", alpha=0.8)

ax2.set_xlabel("Task")
ax2.set_ylabel("Success Rate")
ax2.set_title("3-Way Comparison: Success Rate by Task")
ax2.set_xticks(x)
ax2.set_xticklabels(task_names)
ax2.legend()
ax2.set_ylim(0, 1.0)
ax2.grid(True, alpha=0.3, axis="y")

plt.tight_layout()
plt.savefig(f"{OUTPUT_BASE}/training_comparison.png", dpi=150, bbox_inches="tight")
plt.show()
print(f"Saved to {OUTPUT_BASE}/training_comparison.png")

## 8. Upload best checkpoint to HuggingFace Hub

Save the fine-tuned model so we can load it in the introspection notebook.

In [ ]:
!huggingface-cli login

In [ ]:
import os
HF_USER = os.environ.get("HF_USER", "YOUR_USERNAME")

!huggingface-cli upload {HF_USER}/smolvla-metaworld-finetuned \
    {OUTPUT_BASE}/finetuned/checkpoints/last/pretrained_model

!huggingface-cli upload {HF_USER}/smolvla-metaworld-scratch \
    {OUTPUT_BASE}/scratch/checkpoints/last/pretrained_model

print(f"\nModels uploaded to:")
print(f"  https://huggingface.co/{HF_USER}/smolvla-metaworld-finetuned")
print(f"  https://huggingface.co/{HF_USER}/smolvla-metaworld-scratch")